<table style="width: 100%;"> <tr> <td style="width: 20%; vertical-align: top;"> <img src="https://upload.wikimedia.org/wikipedia/commons/archive/f/fb/20161010213812%21Escudo-UdeA.svg" alt="UdeA" height="150px"> </td> <td style="width: 80%; padding-left: 20px;"> <strong style="font-size: 30px;">TRANSFORMACIÓN DEL CONJUNTO DE DATOS PARA EL DESARROLLO DE UN SISTEMA DE RECOMENDACIÓN DE PELÍCULAS - PROYECTO MARKETING</strong><br><br> <span style="font-size: 30px;"> Luisa Fernanda Alzate Cuartas <br> Juan Camilo Henao Caro<br>Isabella Mendez Hoyos<br> Fernando Antonio Piñeres Ramos </span> </td> </tr> </table>

#### LIBRERÍAS REQUERIDAS

In [1]:
#Conexión de Google Colab con Drive.
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/marketing/Marketing/data')

Mounted at /content/drive


In [2]:
import pandas as pd # Manipulación y el analísis de datos.
import sqlite3 as sql # Interactuar con base de datos SQL usando Python.
from sklearn.preprocessing import MultiLabelBinarizer #Transformación entre iterables de iterables y un formato multietiqueta.
import shutil # Ayuda a automatizar el proceso de copia y eliminación de archivos y directorios.

#### CONEXIÓN CON LA BASE DE DATOS

In [103]:
#Copia de la base de datos original
db_movies = '/content/drive/MyDrive/marketing/Marketing/data/db_movies'
db_movies_copy = '/content/drive/MyDrive/marketing/Marketing/data/db_movies_copy.db'
shutil.copy(db_movies, db_movies_copy)

'/content/drive/MyDrive/marketing/Marketing/data/db_movies_copy.db'

In [104]:
#Establece una conexión a la base de datos SQLite ubicada en "content/db_movies"
conexion = sql.connect(db_movies_copy)

# Crear un cursor a partir de la conexión, que permita ejecutar comando SQL sobre la base de datos.
cur = conexion.cursor()

In [64]:
# Ejecuta una consulta SQL que obtiene los nombres de todas las tablas existentes en la base de datos.
cur.execute("SELECT name FROM sqlite_master where type='table' ")

# Recupera todos los resultados de la consulta anterior en forma de lista de tuplas.
cur.fetchall()

[('ratings',), ('movies',)]

#### EXPLORACIÓN DE LAS TABLAS

##### TABLA MOVIES

In [65]:
# Ejecuta la consulta SQL "SELECT * FROM movies" sobre la base de datos y carga los resultados en un DataFrame de pandas.
table_movies = pd. read_sql_query("SELECT * FROM movies", conexion)
table_movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation





```
# Transformaciones requeridas en la tabla movies
  - Separación del nombre de la película y el año de estreno.
  - Separación de la columna de género en columnas binarias.
  Columna genres contiene múltiples géneros por cada fila.
```





In [66]:
# Usamos una expresión regular para separar el título y el año
table_movies['año'] = table_movies['title'].str.extract(r'\((\d{4})\)', expand=False)
table_movies['titulo'] = table_movies['title'].str.replace(r'\s?\(\d{4}\)', '', regex=True)

table_movies.drop('title', axis=1, inplace=True)

table_movies.head()



,movieId,genres,año,titulo
0,1,Adventure|Animation|Children|Comedy|Fantasy,1995,Toy Story
1,2,Adventure|Children|Fantasy,1995,Jumanji
2,3,Comedy|Romance,1995,Grumpier Old Men
3,4,Comedy|Drama|Romance,1995,Waiting to Exhale
4,5,Comedy,1995,Father of the Bride Part II


In [67]:
# Separar los géneros en listas
table_movies['genres'] = table_movies['genres'].str.split('|')

# Binarizar los géneros
mlb = MultiLabelBinarizer()
generos_bin = mlb.fit_transform(table_movies['genres'])

# Crear DataFrame con los géneros binarizados
df_generos = pd.DataFrame(generos_bin, columns=mlb.classes_, index=table_movies.index)


# Unir los géneros binarizados al DataFrame original
table_movies = pd.concat([table_movies, df_generos], axis=1)

#Eliminar la columna genres
table_movies.drop('genres', axis=1, inplace=True)
table_movies.drop('(no genres listed)', axis=1, inplace=True)

# Mostrar las primeras filas del resultado
table_movies

,movieId,año,titulo,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1995,Toy Story,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1995,Jumanji,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,1995,Grumpier Old Men,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,1995,Waiting to Exhale,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,5,1995,Father of the Bride Part II,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,193581,2017,Black Butler: Book of the Atlantic,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9738,193583,2017,No Game No Life: Zero,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9739,193585,2017,Flint,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9740,193587,2018,Bungo Stray Dogs: Dead Apple,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [80]:
table_movies.isnull().sum()

,0
movieId,0
año,13
titulo,0
Action,0
Adventure,0
Animation,0
Children,0
Comedy,0
Crime,0
Documentary,0


In [78]:
peliculas_sin_genero = table_movies[
    table_movies.drop(['movieId', 'año', 'titulo'], axis=1).sum(axis=1) == 0
][['movieId', 'titulo', 'año']]

peliculas_sin_genero

,movieId,titulo,año
8517,114335,La cravate,1957
8684,122888,Ben-hur,2016
8687,122896,Pirates of the Caribbean: Dead Men Tell No Tales,2017
8782,129250,Superfast!,2015
8836,132084,Let It Be Me,1995
8902,134861,Trevor Noah: African American,2013
9033,141131,Guardians,2016
9053,141866,Green Room,2015
9070,142456,The Brand New Testament,2015
9091,143410,Hyena Road,NaN


hay 34 peliculas sin genero y de esas 34 en 9 el año era nulo por lo que opto por eliminar estos registros de la tabla_movies

In [81]:
table_movies.drop(table_movies[table_movies.drop(['movieId', 'año', 'titulo'], axis=1).sum(axis=1) == 0].index, inplace=True)


In [84]:
# Ver las observaciones con géneros asignados pero sin año
peliculas_sin_año = table_movies[table_movies['año'].isna()]
display(peliculas_sin_año)


,movieId,año,titulo,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
6059,40697,NaN,Babylon 5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
9031,140956,NaN,Ready Player One,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
9179,149334,NaN,Nocturnal Animals,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9367,162414,NaN,Moonlight,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [85]:
# Eliminar las películas con año nulo
table_movies.dropna(subset=['año'], inplace=True)

In [86]:
table_movies

,movieId,año,titulo,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1995,Toy Story,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1995,Jumanji,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,1995,Grumpier Old Men,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,1995,Waiting to Exhale,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,5,1995,Father of the Bride Part II,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,193581,2017,Black Butler: Book of the Atlantic,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9738,193583,2017,No Game No Life: Zero,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9739,193585,2017,Flint,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9740,193587,2018,Bungo Stray Dogs: Dead Apple,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [89]:
table_movies['titulo'].duplicated()

,titulo
0,False
1,False
2,False
3,False
4,False
...,...
9737,False
9738,False
9739,False
9740,False


In [92]:
# Filtrar las filas donde el título está duplicado
titulos_duplicados = table_movies[table_movies['titulo'].duplicated(keep=False)]
display(titulos_duplicados.sort_values('titulo'))


,movieId,año,titulo,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
7332,77846,1997,12 Angry Men,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
905,1203,1957,12 Angry Men,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9051,141844,1971,12 Chairs,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9046,141816,1976,12 Chairs,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
777,1019,1954,"20,000 Leagues Under the Sea",0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5473,26198,1968,"Yours, Mine and Ours",0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9258,156553,2015,Zoom,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6266,47384,2006,Zoom,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4114,5899,1964,Zulu,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [93]:
duplicados_titulo_anio = table_movies[table_movies.duplicated(subset=['titulo', 'año'], keep=False)]
display(duplicados_titulo_anio.sort_values(['titulo', 'año']))


,movieId,año,titulo,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
4169,6003,2002,Confessions of a Dangerous Mind,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,1,0,0
9106,144606,2002,Confessions of a Dangerous Mind,0,0,0,0,1,1,0,...,0,0,0,0,0,1,0,1,0,0
650,838,1996,Emma,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
5601,26958,1996,Emma,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
5854,32600,2004,Eros,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9135,147002,2004,Eros,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2141,2851,1980,Saturn 3,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
9468,168358,1980,Saturn 3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
5931,34048,2005,War of the Worlds,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
6932,64997,2005,War of the Worlds,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [95]:
pd.set_option('display.max_rows', None)  # Esto mostrará todas las filas

# Filtramos las filas duplicadas por 'titulo' y 'año'
duplicados_exactos = table_movies[table_movies.duplicated(subset=['titulo', 'año'], keep=False)]

# Mostramos todos los duplicados ordenados por 'titulo' y 'año'
display(duplicados_exactos.sort_values(['titulo', 'año']))


,movieId,año,titulo,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
4169,6003,2002,Confessions of a Dangerous Mind,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,1,0,0
9106,144606,2002,Confessions of a Dangerous Mind,0,0,0,0,1,1,0,...,0,0,0,0,0,1,0,1,0,0
650,838,1996,Emma,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
5601,26958,1996,Emma,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
5854,32600,2004,Eros,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9135,147002,2004,Eros,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2141,2851,1980,Saturn 3,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
9468,168358,1980,Saturn 3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
5931,34048,2005,War of the Worlds,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
6932,64997,2005,War of the Worlds,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [105]:
# Guardar el DataFrame con las transformaciones en la base de datos de la tabla movies.
table_movies.to_sql('movies', conexion, if_exists='replace', index=False)

#Ayuda a guardar los cambios
conexion.commit()

##### TABLA RATINGS

In [58]:
# Ejecuta la consulta SQL "SELECT * FROM ratings" sobre la base de datos y carga los resultados en un DataFrame de pandas.
table_ratings = pd.read_sql_query("SELECT * FROM ratings", conexion)
table_ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352




```
# Transformaciones requeridas para la tabla rating
 - La columna timestamp almacena información de fecha y hora,
 incluyendo el año, mes, día, hora, minuto y segundo, se puede separar
 para hacer consultas.
```



In [59]:
#Transformación de la columna timestampo por fecha y hora.
table_ratings['timestamp'] = pd.to_datetime(table_ratings['timestamp'], unit='s')
table_ratings['day'] = table_ratings['timestamp'].dt.day
table_ratings['month'] = table_ratings['timestamp'].dt.month

table_ratings['hour'] = table_ratings['timestamp'].dt.hour
table_ratings['year'] = table_ratings['timestamp'].dt.year

if 'date' in table_ratings.columns:
  table_ratings = table_ratings.drop('date', axis=1)

#Eliminar la columna timestampo
table_ratings.drop(columns=['timestamp'], inplace=True)

# Mostrar el DataFrame actualizado
table_ratings


,userId,movieId,rating,day,month,hour,year
0,1,1,4.0,30,7,18,2000
1,1,3,4.0,30,7,18,2000
2,1,6,4.0,30,7,18,2000
3,1,47,5.0,30,7,19,2000
4,1,50,5.0,30,7,18,2000
...,...,...,...,...,...,...,...
100831,610,166534,4.0,3,5,21,2017
100832,610,168248,5.0,3,5,22,2017
100833,610,168250,5.0,8,5,19,2017
100834,610,168252,5.0,3,5,21,2017


In [99]:
# Filtrar ratings para que solo incluyan movieId que están en table_movies
table_ratings = table_ratings[table_ratings['movieId'].isin(table_movies['movieId'])]
table_ratings.shape

(100781, 7)

In [100]:
# Guardar el DataFrame con las transformaciones en la base de datos de la tabla ratings.
table_ratings.to_sql('ratings', conexion, if_exists='replace', index=False)

#Ayuda a guardar los cambios
conexion.commit()

In [101]:
#Cierre de la conexión con la base de datos.
conexion.close()